In [1]:
# Importing the libraries
import pandas as pd
from selenium import webdriver 
 
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Chrome() # creating an instance

In [6]:
urls = {'British Gas': 'https://www.reviews.io/company-reviews/store/british-gas', 
       'E.on': 'https://www.reviews.io/company-reviews/store/eon',
        'Npower': 'https://www.reviews.io/company-reviews/store/npower', 
        "EDF Energy": 'https://www.reviews.io/company-reviews/store/edf-energy1', 
    "Octopus Energy": 'https://www.reviews.io/company-reviews/store/octopus-energy'}

In [4]:
# creating a function that will help in scraping (using findAll)
def locator(class_name):
    time.sleep(2)
    vals = driver.find_elements(By.CLASS_NAME, class_name)
    return [i.text.strip() for i in vals]

In [5]:
def get_rating():
    # It will wait until the element is clickable
    time.sleep(2)
    review_stars = driver.find_elements(By.CLASS_NAME, 'Review__overallStars') # Getting the review starts
    time.sleep(1)
    html_code = [driver.execute_script("return arguments[0].innerHTML;", x) for x in review_stars] # getting the HTML code
    ratings = [score.count('stars__icon--100"') for score in html_code]
    return [1 if rate == 0 else rate for rate in ratings]

In [13]:
dataset = {'Company': [], 'Reviewer Verification Status': [], 'Date': [], 'Rating': [], 'Review': []}

for comp, url in urls.items():
    website = driver.get(url)
    while True:
        # Gets the dataset
        reviewer_verification = locator("BadgeElement__text")
        date_posted = locator('Review__dateSource')
        review = locator('Review__body')
        rating = get_rating()

        # Appending the data to the datasets
        dataset['Reviewer Verification Status'].extend(reviewer_verification)
        dataset['Date'].extend(date_posted)
        dataset['Review'].extend(review)
        dataset['Rating'].extend(rating)
        dataset['Company'].extend([comp] * len(date_posted))

        try:
            next_keys = driver.find_element(By.LINK_TEXT, '»')
            next_keys.click() #next keys
        except:
            break
        

In [15]:
# Saving the data to a dataframe
dataframe_energy = pd.DataFrame(dataset)

In [16]:
dataframe_energy.drop_duplicates(inplace=True)

In [17]:
# Dataframe information
dataframe_energy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4153 entries, 0 to 4195
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Company                       4153 non-null   object
 1   Reviewer Verification Status  4153 non-null   object
 2   Date                          4153 non-null   object
 3   Rating                        4153 non-null   int64 
 4   Review                        4153 non-null   object
dtypes: int64(1), object(4)
memory usage: 194.7+ KB


In [18]:
# Replacing the index
dataframe_energy.reset_index(drop=True, inplace=True)

In [19]:
# Saving the dataset to csv
dataframe_energy.to_csv('Energy-Data.csv', index=False)